In [ ]:
import os
import sys

sys.path.append('..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from AlphaGNN import Alpha
from AMPParams import AMPParams
from GraphBuilder import GraphBuilder
from HybridFF import HybridFF
from Constants import H_TO_KJ, BOHR_TO_ANGSTROM, KC
from Utilities import calculate_d3_dimer, show_results

import Electrostatics
import Polarization

import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cdist

In [ ]:
builder = GraphBuilder()
model_params = AMPParams()
hybrid_ff = HybridFF(pol0=False)

In [ ]:
DATA = np.load('data_S66x8.npy', allow_pickle=True).item()

In [ ]:
predictions, references = [], []
for SYSTEM in DATA:
    elements_1, elements_2 = DATA[SYSTEM]['elements_monomer_1'], DATA[SYSTEM]['elements_monomer_2']
    coords_1, coords_2 = DATA[SYSTEM]['coordinates_monomer_1'], DATA[SYSTEM]['coordinates_monomer_2']
    distance_matrix = cdist(coords_1, coords_2).astype(np.float32)[None]
    # Construct molecular graphs (can be constructed once for each molecular topology)
    graph_1 = builder.from_coords(coords_1, elements_1)
    graph_2 = builder.from_coords(coords_2, elements_2)  
    coords_1, coords_2 = coords_1[None], coords_2[None]
    # Predict atomic multipoles (geometric graphs)
    graph, _ = model_params.predict(coords_1, elements_1)
    monos_1, dipos_1, quads_1, ratios_1 = graph.monos, graph.dipos, graph.quads, graph.ratios
    graph, _ = model_params.predict(coords_2, elements_2)
    monos_2, dipos_2, quads_2, ratios_2 = graph.monos, graph.dipos, graph.quads, graph.ratios
    multipoles = (monos_1, monos_2, dipos_1, dipos_2, quads_1, quads_2)
    alpha_coeffs_1, alpha_coeffs_2 = hybrid_ff.Alpha(graph_1, 1)[None], hybrid_ff.Alpha(graph_2, 1)[None]
    alphas_1, alphas_2 = alpha_coeffs_1 * ratios_1, alpha_coeffs_2 * ratios_2
    # Calculate interaction terms
    V_esp, F1, F2 = Electrostatics.esp_dimer(coords_1, coords_2, distance_matrix, multipoles, with_field=True)    
    V_pol, mu_ind_1, mu_ind_2 = Polarization.ind_dimer(coords_1, coords_2, alphas_1, alphas_2, F1, F2)
    V_D3 = calculate_d3_dimer(coords_1, coords_2, elements_1, elements_2)
    V_ana = hybrid_ff.ANA(graph_1, graph_2, coords_1, coords_2, distance_matrix, multipoles, (mu_ind_1, mu_ind_2), coords_1.shape[0])
    V_total = V_esp + V_pol + V_D3 + V_ana
    V_ref = DATA[SYSTEM]['ref_energy']
    predictions.append(V_total)
    references.append(V_ref)
references, predictions = np.hstack(references), np.hstack(predictions)

In [ ]:
show_results(references, predictions, show_plot=True)